## 序言
1. 隐马尔可夫模型不符合语言规律，且提升隐马尔可夫模型的阶数并不能提升分词的准确率；
2. 线性模型应运而生，线性模型由两部分组成：1.提取特征的特征函数；2.以及相应的权重向量

## 5.1 分类问题
### 5.1.1 定义
### 5.1.2 应用
1. 文本分类
2. 关键词提取：判断每个单词是否属于关键词
3. 指代小姐：代词和实体判断是否存在指代关系
4. 语言模型：每个单词作为一个类别，给定上文预测下文要出现的单词

## 5.2 线性分类模型与感知机算法
线性模型：一条线性的直线或高维平面将数据一分为二，由特征函数$\;\phi\;$，和特征权重向量$\;\omega\;$组成
### 5.2.1 特征向量与样本空间
将样本表示为向量，描述样本特征的向量称为特征向量，构造特征向量的过程称为特征提取；提取特征的函数称为特征函数

### 5.2.2 决策边界与分离超平面
分离超平面方程，又称决策边界方程：
$$\sum_{i=1}^D \omega_i x_i + b = 0$$
使用决策边界方程进行最终决策：
$$\hat{y} = sign(\omega \cdot x)=\lbrace_{1, \omega \cdot x >0}^{-1, \omega \cdot x \le 0}$$

### 5.2.3 感知机算法
在训练集上运行多个迭代，每次读入一个样本，进行预测，将预测结果与正确答案对比，计算误差更新模型参数。
- *感知机算法*

    （1）读入训练样本，执行预测
    （2）如果预测值不等于实际值，更新参数
    
### 5.2.5 投票感知机和平均感知机
1. 投票感知机：将每次迭代的模型和准确率保留。预测时每个模型给出自己的结果，再乘以准确率加权平均值最为最终结果，计算和存储的开销大
2. 平均感知机：取多个模型参数的平均，即所有迭代参数的平均值，较少内存开销，易实现

## 5.3 感知机人名性别分类
### 5.3.1 特征提取
1. 去掉姓氏，名中的每个字都是一个特征，性别是标签，所有测试集中的每个名都是一个特征
2. 感知算法实现
3. 模型训练

# hanlp 实现

In [20]:
import os
import zipfile
from pyhanlp import *
from pyhanlp.static import download, HANLP_DATA_PATH, remove_file
# from mtests.test_utility import ensure_data  # 具体如下

In [21]:
def test_data_path():
    """
    获取测试数据路径
    """
    # HANLP_DATA_PATH 为hanlp保存数据的路径
    data_path = os.path.join(HANLP_DATA_PATH, 'test')
    if not os.path.isdir(data_path):
        os.mkdir(data_path)
    return data_path


# from test.test_utility import ensure_data
def ensure_data(data_name, data_url):
    """
    返回册书数据的存储路径
    """
    root_path = test_data_path()
    dest_path = os.path.join(root_path, data_name)
    if os.path.exists(dest_path):
        return dest_path
    if data_url.endswith('.zip'):
        dest_path += '.zip'
    # 根据文件目录下
    download(data_url, dest_path)
    
    # 解压文件，手动下载文件至目录中
    if data_url.endswith('.zip'):
        with zipfile.ZipFile(dest_path,'r') as archive:
            archive.extractall(root_path)
        remove_file(dest_path)
        dest_path = dest_path[:len('.zip')]
    return dest_path

In [22]:
# 获取数据集
cnname = ensure_data('cnname', 'http://file.hankcs.com/corpus/cnname.zip')
TRAINING_SET = os.path.join(cnname, 'train.csv')   # 训练数据集
TESTING_SET = os.path.join(cnname, 'test.csv')     # 测试数据集

# 加载感知机分类器
PerceptronNameGenderClassifier = JClass('com.hankcs.hanlp.model.perceptron.PerceptronNameGenderClassifier')

In [23]:
def run_classifier(averaged_perceptron):
    print('=====%s=====' % ('平均感知机' if averaged_perceptron else '朴素感知机'))
    clf = PerceptronNameGenderClassifier()
    print('训练集准确率：', clf.train(TRAINING_SET, 10, averaged_perceptron))
    model = clf.getModel()
    print('特征数量', len(model.parameter))
    for name in "赵建军", "沈雁冰", "陆雪琪", "李冰冰":
        print('%s=%s' % (name, clf.predict(name)))
    print('测试集准确率：', clf.evaluate(TESTING_SET))

run_classifier(False)
run_classifier(True)

In [26]:
clf = PerceptronNameGenderClassifier()
clf.train(TRAINING_SET, 10, False)

## 使用sklearn中的感知机

In [28]:
import pandas as pd
from sklearn.linear_model import Perceptron

# 如何进行特征提取

## 平均感知机词性标注 